In [15]:
import numpy as np
import matplotlib.pyplot as plt
from pygsp import graphs, filters, plotting
import pygsp as pg
from numpy.random import default_rng
from numpy.random import randint
from random import sample
import random
import cvxpy as cp

plotting.BACKEND = 'matplotlib'
plt.rcParams['figure.figsize'] = (10, 5)

In [16]:
def createGraph(Nodes,i):
    graphType = randint(0, 6, 1)
    while True:
        s = np.array(sample(range(1, 10000), 1))
        if graphType==0:
            G = pg.graphs.BarabasiAlbert(N = Nodes,seed = s)
            typeName = 'BarabasiAlbert'
        elif graphType == 1:
            G = pg.graphs.RandomRegular(N = Nodes,seed = s)
            typeName = 'RandomRegular'
        elif graphType == 2:
            G = pg.graphs.DavidSensorNet(N = Nodes,seed = s)
            typeName = 'DavidSensorNet'
        elif graphType == 3:
            G = pg.graphs.Community(N = Nodes,Nc = 3,seed = s)
            typeName = 'Community'
        elif graphType == 4:
            G = pg.graphs.ErdosRenyi(N = Nodes,p = 0.1,seed = s)
            typeName = 'ErdosRenyi'
        else:
            G = pg.graphs.Sensor(N=Nodes,seed = s)
            typeName = 'Sensor'
            
        if (G.is_connected() and not(G.is_directed())):
            break
            
    print(typeName)
           
    G.compute_fourier_basis()
    W = np.random.uniform(size=(2,1))
    x =  G.U[:,1:3]@W
    x = x/np.max(np.abs(x))
    eigenvalue = G.e
    print(eigenvalue[1:4])

    return G,x

In [17]:
def computeMainSignal(signals,k,N):
    x = np.zeros((N,1))
    for i in range(k):
        x = x + signals[:,i].reshape(N,1)
    return x

In [18]:
def computeSmoothness(signals,graphs,k):
    smoothness = []
    sumOverSmoothness = 0
    for i in range(k):
        smoothness.append(signals[:,i].T@graphs[i].L@signals[:,i])
        sumOverSmoothness = sumOverSmoothness + smoothness[i]
    return sumOverSmoothness,smoothness

In [19]:
def decomposeGraphSignal(mainSignal,signals,graphs,k,N):    
    # Create A , H
    w = np.arange(1,40*(N-1),40)
    H = np.diag(np.tile(w, (1, k))[0])
    for i in range(k):
        G = graphs[i]
        A_init = G.U[:, 1:]
        if i==0:
            A = A_init
        else:
            A = np.hstack((A, A_init))

    # solve by CVX
    Y = cp.Variable(((N-1)*k,1))
    objective = cp.Minimize(cp.quad_form(Y, H))  
    # equality 
    constraints = [A @ Y == mainSignal]
    prob = cp.Problem(objective, constraints)
    result = prob.solve(verbose=True)
    Y_hat = np.vstack((np.zeros((1,k)),Y.value.reshape(N-1,k,order='F')))

    # IGFT
    X_hat = np.zeros((N,k))
    for i in range(k):
        G = graphs[i]
        X_hat[:,i] = (G.U @ Y_hat[:,i])
    
    return X_hat

In [20]:
def calculateSNR(est_signals,signals,k):
    SNR = np.zeros((1,k))
    for i in range(k):
        SNR[0,i]=(10*np.log10(np.linalg.norm(signals[:,i])**2/np.linalg.norm(signals[:,i]-est_signals[:,i])**2))
    return SNR

In [21]:
N = 100   # number of nodes
k = 5     # number of graphs
np.random.seed(0)
random.seed(0)
iteration = 200
av_snr = np.zeros((1,k))

for itera in range(1,iteration+1):
    print('############################################# experiment = ',itera,' ########################################')
    # create graph and graph signals
    graph_list = []
    signals = np.zeros((N,k))
    for i in range(k):
        G , x = createGraph(N,i)
        graph_list.append(G)
        signals[:,[i]] = x

    # calculate the main signal : x = x1+x2+...+x4
    mainSignal = computeMainSignal(signals,k,N)
    mainSignal = mainSignal - np.mean(mainSignal)

    # decompose graph signal
    estimated_signals = decomposeGraphSignal(mainSignal,signals,graph_list,k,N)

    # calculate graph signals' smoothness
    sumOverSmoothness,smoothness = computeSmoothness(signals,graph_list,k)
    print('smoothness:')
    print(smoothness)
    print('sum over smoothness:',sumOverSmoothness)

    # calculate estimated graph signals' smoothness
    sumOverSmoothness_est,smoothness_est = computeSmoothness(estimated_signals,graph_list,k)
    print('smoothness_est:')
    print(smoothness_est)
    print('sum over smoothness_est:',sumOverSmoothness_est)

    # calculate SNRs
    SNRs = calculateSNR(estimated_signals,signals,k)
    print('SNR:',SNRs)
    av_snr = av_snr + SNRs/iteration

    # compute objective values
    objectiveValue = sumOverSmoothness
    objectiveValue_est = sumOverSmoothness_est
    print('objectiveValue:',objectiveValue)
    print('objectiveValue_est:',objectiveValue_est)

    # compute error
    error = np.linalg.norm(mainSignal-estimated_signals@np.ones((k,1)))
    print('error:',error)
    
    
print('av_snr: ',av_snr)
print('total_av_snr: ',np.mean(av_snr))

2021-12-03 15:09:46,995:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:09:47,079:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979


############################################# experiment =  1  ########################################
ErdosRenyi
[3.15457246 3.32524664 3.37007353]


2021-12-03 15:09:47,171:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:09:47,267:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979


Community
[0.26422716 0.37190397 0.44088793]
Sensor
[0.13262437 0.30306048 0.39936753]
BarabasiAlbert
[0.01641036 0.02278579 0.03819425]
BarabasiAlbert
[0.02937406 0.03658478 0.04724493]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 495, constraints m = 100
          nnz(P) + nnz(A) = 49888
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res  

2021-12-03 15:09:48,255:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:09:48,356:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979


...

2021-12-03 15:18:36,617:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:18:36,716:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:18:36,842:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:18:36,957:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:18:37,069:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:18:37,191:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:18:37,289:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979


Community
[0.22004174 0.29555463 0.39898128]
BarabasiAlbert
[0.02391513 0.02887684 0.04593052]

2021-12-03 15:18:37,602:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:18:37,721:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979


2021-12-03 15:18:37,823:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:18:37,928:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:18:38,018:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979
2021-12-03 15:18:38,121:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 2.23606797749979


Community
[0.31475913 0.35845837 0.45049228]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 495, constraints m = 100
          nnz(P) + nnz(A) = 49935
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   2.42e+00   8.55e+04   1.00e-01   3.47e-02s
  50   6.9425e+02   3.24e-09   1.15e-06   1.00e-01   5.62e-02